In [ ]:
import numpy as np
import numpy.linalg as LA
from scipy.linalg import expm
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from sympy import symbols, Matrix, latex


In [ ]:
c1 = 2
c2 = 1

M = np.array([[0, 1], [-2, -1]])
x0 = np.array([1, 0])
t = np.linspace(0, 1, 10)
t_arr = t.reshape((10, 1))

N = len(x0.shape)
n_t = len(t)

eva, V = LA.eig(M)
coeff = np.dot(LA.inv(V), x0)

sol = np.diag((N, N))

In [ ]:
v_sol = np.exp(eva * t_arr)
n, m = v_sol.shape

x_sol = np.dot(V * np.diag((N, N)) * coeff, v_sol.reshape(m, n))

In [ ]:
V * np.diag((N, N)) * coeff * v_sol[3]

In [ ]:
M = np.array([[0, 1], [-2, 0]])
la, v = LA.eig(M)

In [ ]:
a, b, c, d = symbols('a b c d', positive=True)
om, de = symbols('omega0 delta', positive=True)
A = Matrix([[-2, -4, 2], [-2, 1, 2], [4, 2, 5]])
A = Matrix([[0, 1], [-om**2, -2*de]]).subs(de, om)
# A = Matrix([[a, b], [c, d]])
for ev, mult, v in A.eigenvects():
    display(ev)
    display(v)

In [ ]:
def de(t, x, M):
    return np.dot(M, x)


def exact_sol(t, x0, M):
    eva, eve = LA.eig(M)
    coeff = np.dot(LA.inv(eve), x0)

    sol = np.zeros((len(x0), len(t)))

    for i, ev in enumerate(eva):
        for j, comp in enumerate(eve[:, i]):
            sol[j] += np.real(comp * coeff[i] * np.exp(ev * t))

    return sol
    
def y(t, y0, om0, de):
    om = np.sqrt(om0**2-de**2) 
    return y0*np.exp(-de*t)*(np.cos(om*t)+de/om*np.sin(om*t))

def yappr(t, y0, om0, de):
    om = np.sqrt(om0**2-de**2) 
    return y0*np.exp(-de*t)*np.cos(om*t)
                             
def vy(t, y0, om0, de):
    om = np.sqrt(om0**2-de**2) 
    return -y0*om0**2/om*np.exp(-de*t)*np.sin(om*t)

def y2(t, v0, om0, de):
    om = np.sqrt(om0**2-de**2) 
    return v0/om*np.exp(-de*t)*np.sin(om*t)
                             
def vy2(t, v0, om0, de):
    om = np.sqrt(om0**2-de**2) 
    return v0/om*np.exp(-de*t)*(om*np.cos(om*t)-de*np.sin(om*t))

def y_oc(t, y0, om0, de):
    de_f = de+np.sqrt(de**2-om0**2)
    de_s = de-np.sqrt(de**2-om0**2)
    return y0*(+de_f*np.exp(-de_s*t)-de_s*np.exp(-de_f*t))/(de_f-de_s)

def vy_oc(t, y0, om0, de):
    de_f = de+np.sqrt(de**2-om0**2)
    de_s = de-np.sqrt(de**2-om0**2)
    return y0*om0**2*(np.exp(-de_f*t)-np.exp(-de_s*t))/(de_f-de_s)

def y_oc2(t, v0, om0, de):
    de_f = de+np.sqrt(de**2-om0**2)
    de_s = de-np.sqrt(de**2-om0**2)
    return v0/om0**2*de_s*de_f/(de_f-de_s)*(np.exp(-de_s*t)-np.exp(-de_f*t))

def vy_oc2(t, v0, om0, de):
    de_f = de+np.sqrt(de**2-om0**2)
    de_s = de-np.sqrt(de**2-om0**2)
    return v0*(de_f*np.exp(-de_f*t)-de_s*np.exp(-de_s*t))/(de_f-de_s)

def y_c(t, y0, v0, de):
    return (y0+(de*y0+v0)*t)*np.exp(-de*t)

def vy_c(t, y0, v0, de):
    return (v0-(de**2*y0+de*v0)*t)*np.exp(-de*t)

a = 0
b = 1
c = -4 + 0j
d = -4 + 0j

tau = a + d
det = a*d - b*c
disc = tau**2 - 4*det
la1 = (-tau+np.sqrt(disc))/2
la2 = (-tau-np.sqrt(disc))/2

M = np.array([[a, b], [c, d]])

x0 = np.array([1, 0])

tmin = 0
tmax = 5
N = 1000
t_eval = np.linspace(tmin, tmax, N)

sol = solve_ivp(de, [tmin, tmax], x0, args=(M,), t_eval=t_eval)
y_num = sol.y[0]
vy_num = sol.y[1]

R, J = exact_sol(t_eval, x0, M)

print(f'trace: {tau}, determinant: {det}, discriminant: {disc}')
print(f'eigenvalues {la1}, {la2}')

# plt.plot(t_eval, R, label='$y(t)$')
# plt.plot(t_eval, J, label='$v_y(t)$')
# plt.plot(t_eval, y_num, label='$y(t) (numerical)$')
# plt.plot(t_eval, vy_num, label='$vy(t) (numerical)$')
# plt.plot(t_eval, y_oc(t_eval, x0[0], np.sqrt(-c), -d/2), label='ex')
# plt.plot(t_eval, y_oc2(t_eval, x0[1], np.sqrt(-c), -d/2), label='ex')
# plt.plot(t_eval, vy_oc(t_eval, x0[0], np.sqrt(-c), -d/2), label='ex')
# plt.plot(t_eval, vy_oc2(t_eval, x0[1], np.sqrt(-c), -d/2), label='ex')
# plt.plot(t_eval, y(t_eval, x0[0], np.sqrt(-c), -d/2))    
# plt.plot(t_eval, yappr(t_eval, x0[0], np.sqrt(-c), -d/2))    
# plt.plot(t_eval, vy(t_eval, x0[0], np.sqrt(-c), -d/2))    
# plt.plot(t_eval, y2(t_eval, x0[1], np.sqrt(-c), -d/2))    
# plt.plot(t_eval, vy2(t_eval, x0[1], np.sqrt(-c), -d/2))    
plt.plot(t_eval, y_c(t_eval, x0[0], x0[1], -d/2), label='$y(t)$')
plt.plot(t_eval, vy_c(t_eval, x0[0], x0[1], -d/2), label='$v_y(t)$')
plt.grid()
plt.xlabel('time')
plt.legend()


In [ ]:
b1, b2, b3 = symbols('b1 b2 b3')
A = Matrix([[1, 1, 0], [0, -1, 1], [1, 0, -1]])
sol = Matrix([1, 1, 1])
(A.inv()*sol)

In [ ]:
np.linalg.eig(M)